# On hurricanes

In [ ]:
# Don't change this cell; just run it.
import numpy as np  # The array library.

import pandas as pd
pd.set_option('mode.copy_on_write', True)

# The OKpy testing system.
from client.api.notebook import Notebook
ok = Notebook('hurricanes.ok')

## Is climate change having an effect on hurricanes?

The US [National Oceanic and Atmospheric Administraition](https://www.noaa.gov)
has a [May 2023 briefing on hurricanes and climate
change](https://sciencecouncil.noaa.gov/wp-content/uploads/2023/05/1.1_SOS_Atlantic_Hurricanes_Climate.pdf).  They say:

> Decreases in aerosol forcing since the 1970s and multidecadal ocean
circulation changes are thought to be contributing to the increased Atlantic
hurricane activity since 1980, though their relative contributions are still
uncertain and with no scientific consensus. While greenhouse gas-induced
warming may have also affected Atlantic hurricane activity, a detectable
greenhouse gas influence on hurricane activity has not been identified with
high confidence.

### The data

Let's have a look at the data.  Happily, the NOAA provide the data on their [hurricane data website](https://www.nhc.noaa.gov/data). 

We are going to look at the most up to date data on Atlantic hurricanes (at
time of writing), stored at
<https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2022-050423.txt>.

### Reading the data

The NOAA give a [detailed description of these
data](https://www.nhc.noaa.gov/data/hurdat/hurdat2-format-atl-1851-2021.pdf).

But first, let's get the text from the data file.

To do that, we will use Python's `pathlib` module, and specifically, the `Path`
class.  See [the pathlib page](https://lisds.github.io/dsip/pathlib.html).
A `Path` object represents a file on disk.  Among other things, it allows us to
read in all the data from the file as text (one long string).

In [ ]:
# Get the constructor to make Path objects.
from pathlib import Path

Here we read the text from a copy of the `hurdat2-1851-2022-050423.txt`. we
have already downloaded from the link above.

In [ ]:
hurdat_path = Path('data/hurdat2-1851-2022-050423.txt')
# Read the whole contents of the file in as one long string.
hurdat_text = hurdat_path.read_text()
# Show the first 200 characters of the contents
hurdat_text[:200]

Notice the `\n` characters in the string above.  These are line end markers. We
can split the text into *lines* by splitting them at these markers.

We could use `hurdat_text.split('\n')` for this, but there's a memorable string
method called `.splitlines` that will do this for us.

In [ ]:
hurdat_lines = hurdat_text.splitlines()
# Show the first 20 lines
hurdat_lines[:20]

As you see here, and as described in the documentation above, the file has the
following format for each storm:

First there is a *header* line. For example, the first header line here is
`'AL011851,            UNNAMED,     14,`.

The header line has three values, separated by commas:

* The storm identifier, e.g. `AL011851`.  The `AL` prefix identifies this as an
  Atlantic storm, and the final four digits are the year of the hurricane.
* The storm name, or `UNNAMED` if it did not have a name.
* The number of *tracking lines* that follow.  The tracking lines give
  information about the storm at the given times.

Next there follow one or more *tracking lines*.  Each line gives the position
and wind speed of the storm, as well as other information.

For example, the first tracking line above starts with `'18510625, 0000,  , HU,
28.0N,  94.8W,  80`.

These are:

* `18510625`: the year, month and day of the observation.
* `0000`: the time of the observation.
* `HU`: the status of the storm (of which more later).  `HU` means the NOAA has
  labeled this storm a hurricane, at this time and day.
* `28.0N` and `94.8W` give the position of the storm in latitude and longitude.
* `80` gives the estimated maximum wind speed, in knots.

### Processing the text file

To process the text file into a Pandas data frame, we have to combine the header information about the storm with the tracking information.  Here's one way of doing that:

In [ ]:
# Run this cell
lines_for_csv = []
for line in hurdat_lines:
    n_commas = line.count(',')  # Count the number 
    if n_commas == 3:   # This must be a header line.
        storm_header = line
        # Go back to the beginning of the loop and continue.
        continue
    # If we got here, this must be a tracking line.
    # Append the header to the tracking line.  Store for later.
    lines_for_csv.append(storm_header + line)

# Show the first 5 processed lines
lines_for_csv[:5]

We also need the column names.  These we got from the description document
linked above.

In [ ]:
col_names = [
    "Storm_ID",
    "Storm_Name",
    "Track_Count",
    "Year_Month_Day",
    "UTC_Hours_Minutes",
    "Record identifier",
    "Status of system",
    "Latitude_NS",
    "Longitude_WE",
    "Maximum sustained wind (in knots)",
    "Minimum Pressure (in millibars)",
    "34 kt wind radii maximum extent in NE quadrant",
    "34 kt wind radii maximum extent in SE quadrant",
    "34 kt wind radii maximum extent in SW quadrant",
    "34 kt wind radii maximum extent in NW quadrant",
    "50 kt wind radii maximum extent in NE quadrant",
    "50 kt wind radii maximum extent in SE quadrant",
    "50 kt wind radii maximum extent in SW quadrant",
    "50 kt wind radii maximum extent in NW quadrant",
    "64 kt wind radii maximum extent in NE quadrant",
    "64 kt wind radii maximum extent in SE quadrant",
    "64 kt wind radii maximum extent in SW quadrant",
    "64 kt wind radii maximum extent in NW quadrant",
    "Radius of Maximum Wind"]

# Make the header line for the CSV file.
header_for_csv = ','.join(col_names)
header_for_csv

In [ ]:
# We can write out the processed file with another Path object.
out_path = Path('data/hurdat_processed.csv')
out_lines = [header_for_csv] + lines_for_csv
out_text = '\n'.join(out_lines)  # Put the lines back together again.
out_path.write_text(out_text)

For reasons that will become clear later, when we read back the data, we need to specify that the `Year_Month_Day` and `UTC_Hours_Minutes` columns should be interpreted as strings rather than numbers.

In [ ]:
hurdat = pd.read_csv(out_path,
                     dtype={'Year_Month_Day': str,
                            'UTC_Hours_Minutes': str})
hurdat.head(20)

### Validation checks

First let's do a correctness check.  The header line gave the number of
tracking measurements for each storm.  In our processed data, the track count
is in the column `Track_Count`. Therefore, for each storm:

* All the `Track_Count` values should be the same and
* The `Track_Count` value should equal the number of rows corresponding to that
  storm.

Let's check that.

First write a function that accepts a series as an argument, and return True
if all the values in the series are the same, and False otherwise.

**Hint** — you may well want to use a function like
[`np.all`](https://numpy.org/doc/stable/reference/generated/numpy.all.html) or
the [Pandas DataFrame `.all`
method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.all.html).

In [ ]:
def all_same(series):
    """ Return True if all the values in the series are the same

    Parameters
    ----------
    series : Series

    Returns
    -------
    same_tf : bool
        True if all the values are the same, False otherwise.
    """
    ...
    return ...

In [ ]:
# Test your function
a_series = pd.Series([10, 10, 10, 10])
assert all_same(a_series) == True
b_series = pd.Series([10, 10, 5, 10])
assert all_same(b_series) == False
c_series = pd.Series(['one', 'one', 'one', 'one'])
assert all_same(c_series) == True

Now create a new Series that has True for the storms where all corresponding rows had the same value for `Track_Count` and False otherwise.

Be careful, the storm names do not identify the storm, because many of them are `'UNNAMED'`.

In [ ]:
storm_same_count = ...
# Show the result
storm_same_count.head(5)

In [ ]:
_ = ok.grade('q_storm_same_count')

Do all the storms have True for that check?  Use code to confirm:

In [ ]:
all_check_out = ...

In [ ]:
_ = ok.grade('q_all_check_out')

Next, make a new data frame with one row per storm, and two columns.  The first
is the `Track_Count` for the corresponding storm, and the second is the number
of rows for that storm, as measured by the number of values in the `Storm_ID`
column.

The first few lines of your table should look like this:

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }
    .dataframe tbody tr th {
        vertical-align: top;
    }
    .dataframe thead th {
        text-align: right;
    }
</style>
<table border=\"1\" class=\"dataframe\">
  <thead>
    <tr style=\"text-align: right;\">
      <th></th>
      <th>Track_Count</th>
      <th>Storm_ID</th>
    </tr>
    <tr>
      <th>Storm_ID</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>AL011851</th>
      <td>14</td>
      <td>14</td>
    </tr>
    <tr>
      <th>AL011852</th>
      <td>45</td>
      <td>45</td>
    </tr>
    <tr>
      <th>AL011853</th>
      <td>1</td>
      <td>1</td>
    </tr>
  </tbody>
</table>
</div>

In [ ]:
# Show the result
storm_counts

Use code to set the variable `counts_match` to True if the counts match  for all storms, in the two columns.

In [ ]:
counts_match = ...

In [ ]:
_ = ok.grade('q_counts_match')

### Cleaning

Now we start to do some cleaning.  First we notice that the values in the
`Storm_Name` column have an lots of spaces at the beginning:

In [ ]:
hurdat['Storm_Name'].iloc[0]

Strip the spaces from the `Storm_Name` column, and replace the current `Storm_Name` column with the stripped values.

In [ ]:
hurdat['Storm_Name'] = ...
# Show the first value.
hurdat['Storm_Name'].iloc[0]

In [ ]:
_ = ok.grade('q_storm_name')

Do we have the same problem for the `Status of system` column?  Investigate. If
so, then fix in the same way.

In [ ]:
#- Investigate the Status of system column.  Fix as necessary.
...

In [ ]:
_ = ok.grade('q_status_of_system')

### Processing dates

We want to track changes in the hurricane data over time.

In order to do this, we need to be get years and maybe months from the information in the data frame.

We will also soon want to be able to get times from the data frame, to select
tracks (measurements) for the accumulated cyclone energy (ACE) (see below).

Do do this, we want to create a new column in `hurdat` called `Datetime`, that
stores the date and time in a special datetime format.

To start this process, we first need a Series that is the concatenation of the
`UTC_Hours_Minutes` column values to the end of the `Year_Month_Day` column
values.  In fact the `Year_Month_Day` values have spaces at the end.  The first
two values of the new series `dt_strings` should be `18510625 0000` and
`18510625 0600`.

By the way, the concatenation we are doing here is why we had to read in these
two columns as strings when we first did `pd.read_csv` on the processed text.

In [ ]:
dt_strings = ...
# Show the first five results.
dt_strings.head()

In [ ]:
_ = ok.grade('q_dt_strings')

Use the `pd.to_datetime` function to convert the `dt_string` values to datetime
values, and put the result in the `Datetime` column of `hurdat`:

In [ ]:
# Run this cell.
hurdat['Datetime'] = pd.to_datetime(dt_strings)
hurdat.head()

Notice you can now get the year of the measurement using the `dt` attribute of
the new `Datetime` series:

In [ ]:
# Run this cell.
hurdat['Year'] = hurdat['Datetime'].dt.year
hurdat['Year']

How many tracks (measurements, rows) qualified as hurricanes for each year?

Hurricane tracks are rows with `HU` in `Status of system`.

In [ ]:
hu_tracks_per_year = ...
                     ...
# Show the result
hu_tracks_per_year

In [ ]:
_ = ok.grade('q_hu_tracks_per_year')

In [ ]:
#- Plot the number of hurricane tracks (measurements) per year.

### Measuring storms

Our next step is to investigate some measure of Atlantic storms, to assess the
NOAA opinion above.

One popular measure of storm severity is [Acculated Cyclone
Energy](https://en.wikipedia.org/wiki/Accumulated_cyclone_energy).

The definitive reference for this measure appears to be [Bell *et al* 2000
— "Climate Assessment for
1999"](https://journals.ametsoc.org/downloadpdf/journals/bams/81/6/1520-0477_2000_81_s1_caf_2_0_co_2.xml).

Quoting from that document:

> The second measure of over- all seasonal activity is referred to as the
hurricane destruction potential (HDP), which is calculated by summing the
squares of the estimated 6-hourly maximum sustained wind speed (${V_{max}}^2$)
for all periods in which the system is a hurricane. This index represents
a single, continuous distribution that implicitly accounts for numbers of
hurricanes, yet also gives more weight to strong systems and long-lasting
systems. A slight modification of the HDP index involves accumulating
${V_{max}}^2$ for all 6-hourly periods in which the system is either a tropical
storm or hurricane, thereby also accounting for the number and duration of
storms while at a tropical storm status. This modified HDP index is referred to
as accumulated cyclone energy (ACE) index (Fig. 28), and is both a physically
and statistically reasonable measure of overall activity during a given
hurricane season.

As you will see from the Wikipedia page above, is also typical to divide the
ACE measure by 10,000 to make the numbers easier to read.

So, to recap, the ACE measure for one storm would involve the following steps:

* Identify the tracking measurements ("tracks") corresponding to the storm.
* From these, select wind-speed measurements measured at 6 hour intervals.   In
  practice these are always measures taken or estimated at 00:00, 06:00, 12:00
  and 18:00.  We will discard measures at other times.
* From these measures, select those where the status of the storm at the
  time qualified it as a storm or a hurricane.  For example, we discard any
  tracks (measurements) where, at the time, the storm was classified as
  a Tropical Depression or similar.
* For the remaining observations, square all the measures of maximum sustained
  wind speed, add these up, and divide by 10,000.

For example, consider the observations for the first storm in the data frame:

In [ ]:
first_storm_rows = hurdat[hurdat['Storm_ID'] == hurdat['Storm_ID'].iloc[0]]
first_storm_rows

Notice that all the observations are at the correct times of 00:00, 06:00, 12:00 and 18:00 except the row labeled 4, measured at 21:00.   We drop that row:

In [ ]:
valid_first = first_storm_rows.drop(index=4)
valid_first

Notice too that, for this first storm, all the tracks were rated as either
`TS` (tropical storm) or 'HU' (hurricane), so they all qualify.  All that
remains is to calculate the ACE score with the wind speeds in the wind speed
column:

In [ ]:
ws_col = "Maximum sustained wind (in knots)"
vmax_vals = valid_first[ws_col]
vmax_vals

Write a function `calc_ace` that accepts a sequence of values as an argument,
and returns the ACE value.

In [ ]:
def calc_ace(v_maxes):
    """ Calculate accumulated cyclone energy from `v_maxes`

    Parameters
    ----------
    v_maxes : array or list
        Sequence of valid maximum sustained wind speeds, in knots.

    Returns
    -------
    ace : float
        Accumulated Cylone Energy
    """
    ...
    return ace

In [ ]:
# Test your function with the values from the first storm.
# It should show a value around 5.
calc_ace(vmax_vals)

In [ ]:
_ = ok.grade('q_calc_ace')

Now for the big reveal.

First — select all the tracks (rows) in `hurdat` for which the following are
true:

* The time of collection is one of 00:00, 06:00, 12:00 or 18:00 and
* The status is one of the storm types (see below), and
* The maximum sustained wind wind speed value is >= 34 (storm intensity)

Here are the types (status values) we'll identify as storms:

In [ ]:
# From https://www.nhc.noaa.gov/data/hurdat/hurdat2-format-atl-1851-2021.pdf
storm_types = (
    'TS', # Tropical cyclone of tropical storm intensity (34-63 knots)
    'HU', # Tropical cyclone of hurricane intensity (> 64 knots)
    'SS', # Subtropical cyclone of subtropical storm intensity (> 34 knots))
)
storm_types

**Hint** - consider using attributes of `hurdat['Datetime'].dt` to check for
the valid times.

In [ ]:
valid_tracks = ...
    ...
# Show the first 20 tracks.
valid_tracks.head(20)

In [ ]:
_ = ok.grade('q_valid_tracks')

Calculate the total ACE per year:

In [ ]:
ace_per_year = ...
# Show the result
ace_per_year

In [ ]:
_ = ok.grade('q_ace_per_year')

In [ ]:
#- Plot the ACE scores per year.
#- This will help you answer the text question at the end.

Now we have the valid tracks and the ACE calculation, we can check our ACE
calculation on some known values.

From the [Wikipedia page on
ACE](https://en.wikipedia.org/wiki/Accumulated_cyclone_energy) we see that
Hurricane Ivan from 2004 had an ACE value of 70.4, and Hurricane Isabel from
2003 had an ACE of 63.3.

Notice that there could be multiple hurricanes called Ivan and Isabel, so you
need the name and the year to identify them.

Calculate the matching values from our data.

Do you get the compatible values with your calculation (within rounding error)?

In [ ]:
ivan_2004_ace = ...
# Show the result
ivan_2004_ace

In [ ]:
isabel_2003_ace = ...
# Show the result
isabel_2003_ace

In [ ]:
_ = ok.grade('q_example_aces')

This one is a fair bit harder.  Count the *number of storms* per year, that
have at least 5 valid tracks.

Your result `n_storms_by_year` should be a series with index Year and values
being the number of *storms* in that year with at least 5 valid tracks.
(Valid means qualifying for the ACE calculation above).

Categorize a storm as having occurred in a year if the first valid measurement is in that year.

In [ ]:
n_storms_by_year = ...
                   ...
# Show the result
n_storms_by_year

In [ ]:
_ = ok.grade('q_n_storms_by_year')

In [ ]:
#- Plot the number of storms
#- This will help you answer the text question following.

### And the answer is

Given the information that you have here, including the NOAA summary linked
above, and any other reading you might like to do, evaluate NOAAs arguments
that lead them to conclude that it is not yet possible to say with high
confidence that rises in greenhouse gases have affected Atlantic hurricane
activity.

This text question counts for 4 times the marks of the code questions.

*Write your answer here, replacing this text.*

## Done.

Congratulations, you're done with the assignment!  Be sure to:

- **run all the tests** (the next cell has a shortcut for that).
- **Save and Checkpoint** from the `File` menu.

In [ ]:
# For your convenience, you can run this cell to run all the tests at once!
import os
_ = [ok.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]